# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 12
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiDi

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 60
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiD

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355079178900                   -1.10    6.1    141ms
  2   -9.356770917343       -2.77       -1.44    1.0   73.5ms
  3   -9.357065870780       -3.53       -2.80    3.9    124ms
  4   -9.357110452401       -4.35       -3.02    5.8    170ms
  5   -9.357110868498       -6.38       -3.15    2.9   91.5ms
  6   -9.357111161609       -6.53       -3.29    1.5   84.1ms
  7   -9.357111355517       -6.71       -3.45    2.1   83.8ms
  8   -9.357111440410       -7.07       -3.60    1.0   73.1ms
  9   -9.357111489694       -7.31       -3.84    1.0   71.0ms
 10   -9.357111503415       -7.86       -4.07    2.4   90.3ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74732821048                   -0.97    6.0    378ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     5
│     1
│     1
│     1
│     1
│     6
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75458285841       -2.14       -1.34    2.1    253ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      3
│      2
│      3
│     10
│      2
│      9
│      1
│      8
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -18.79237921996       -1.42       -1.91    4.8    309ms
  4   -18.79257223327       -3.71       -2.15    4.2    304ms
  5   -18.79259648992       -4.62       -3.31    1.5    226ms
  6   -18.79260562662       -5.04       -3.46    7.9    469ms
  7   -18.79260627467       -6.19       -3.73    1.0    198ms
  8   -18.7

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53798572137                   -0.84    8.9    2.07s
  2   -37.55833070525       -1.69       -1.22    2.5    802ms
  3   -37.55993120671       -2.80       -2.59    7.5    1.25s
┌ Warning: Detected large minimal occupation 1.728520054247905. SCF could be unstable. Try switching to adaptive band selection (`nbandsalg=AdaptiveBands(model)`) or request more converged bands than 29 (e.g. `nbandsalg=AdaptiveBands(model; n_bands_converge=32`)
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:87
  4   -37.56492416489       -2.30       -2.18   15.2    1.80s
  5   -37.56492551344       -5.87       -2.18    4.6    958ms
  6   -37.56493769600       -4.91       -2.24    1.0    753ms
  7   -37.56491125889   +   -4.58       -2.61    2.4    1.30s
  8   -37.56492984272       -4.73       -2.78    1.0    747ms
  9   -37.56493312088       -5.48       -2.82    3

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355128612544                   -1.10    5.9    182ms
  2   -9.356792289073       -2.78       -1.91    1.0   65.3ms
  3   -9.357083750740       -3.54       -2.57    4.2    129ms
  4   -9.357090544960       -5.17       -2.59    4.4    120ms
  5   -9.357111223342       -4.68       -3.67    1.0    111ms
  6   -9.357111417814       -6.71       -3.85    4.8    131ms
  7   -9.357111506024       -7.05       -4.60    1.6    530ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     11
│      3
│     14
│      5
│     19
│      9
│     14
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54386694065                   -0.84    9.6    1.57s
  2   -37.51464570573   +   -1.53       -1.49    3.4    759ms
  3   -10.94619513623   +    1.42       -0.35   10.4    2.40s
  4   -35.98532403038        1.40       -0.87    7.5    1.74s
  5   -37.54100058307        0.19       -1.66    6.8    1.47s
  6   -37.53004360823   +   -1.96       -1.65    1.8    749ms
  7   -37.52158550249   +   -2.07       -1.64    2.2    809ms
  8   -37.56286270212       -1.38       -1.98    1.8    751ms
  9   -37.56305589292       -3.71       -2.17    2.9    930ms
 10   -37.56407774626       -2.99       -2.40    2.4    1.41s
 11   -37.56493721671       -3.07    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.